# Benchmark for bayesian optimization

Warning: By default Botorch maximizes the function. I will need to take the negative of my gt_func

In [1]:
from test_functions import Hart6
from bayes_lib import ExactGPModel, train_hyper_params
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.constraints import Interval
import torch
import matplotlib.pyplot as plt

In [2]:
from scipy.stats import truncnorm
def get_test_x(n, mu=0.5, sigma=0.25, lower=0.0, upper=1.0):
    """ returns a n x d tensor containing random points taking from truncated 
    normal distributions of means mu, and standard deviation sigma (mu and sigma are floats or lists of length d).
    The distribution is truncated between lower and upper bounds.
    for more information on the truncated normal distribution see: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.truncnorm.html
Parameters
==============
n: int
    number of points to return
mu: float or array-like, default: 0.5
    list of means of length d
sigma: float or array-like, default: 0.25
    list of standard deviations (must be the same length as mu)
lower: float, default: 0.0
    lower bound
uper: float, default: 1.0

Returns
==============
tensor of shape (n, d)

    """
    if type(mu) == int or type(mu) == float:
        mu = [mu]
    if type(sigma) == int or type(sigma) == float:
        sigma = [sigma]
    if len(mu) != len(sigma):
        raise ValueError(f"mu and sigma must have the same length, but got len(mu)={len(mu)}, len(sigma)={len(sigma)}")
        
    ndim = len(mu)
    out = []
    for i in range(ndim):
        X = truncnorm(
            (lower - mu[i]) / sigma[i], (upper - mu[i]) / sigma[i], loc=mu[i], scale=sigma[i])
        out.append(torch.from_numpy(X.rvs(n)))

    return torch.stack(out).T.reshape(-1,1,ndim).float()

In [3]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.models.gpytorch import GPyTorchModel
from IPython.display import clear_output
class ExactGPModel(GPyTorchModel, gpytorch.models.ExactGP):
    _num_outputs = 1
    
    def __init__(self, train_x, train_y, likelihood):
        # super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        # self.to(train_x)
        
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
   

In [10]:
# Bayes opt loop
min_iter = 2
max_iter = 20
it = 0
Model = ExactGPModel
gt_func = Hart6(return_negative=True) # objective function
likelihood = GaussianLikelihood(noise_constraint=Interval(0.0,1e-14))


n = int(1e5) # number of random evaluation points of the test function


In [11]:
train_x = torch.rand(6).reshape(-1,6)
train_y = gt_func.f(train_x)
it = 0
best_f = 0.1
rel_tol = 1e-06
test_x = get_test_x(n, mu=[0.5 for i in range(6)],  sigma=[0.25 for i in range(6)])
for i in range(20):
    # Run the forward model with hyperparam opt
    model = Model(train_x, train_y, likelihood)
    train_hyper_params(model, likelihood)

    # Run the aquisition method
    EI = ExpectedImprovement(model, best_f=best_f, maximize=True)
    ei = EI(test_x)
    
#     # Add the suggested point to the training points
    x_new = test_x[ei.argmax().item()]
    y_new = gt_func.f(x_new)
    if y_new.item() > best_f:
        best_f = y_new.item()

    train_x = torch.cat((train_x.reshape(-1,1), x_new.reshape(-1,1))).reshape(-1,6)
    train_y = torch.cat((train_y.reshape(-1,1), y_new.reshape(-1,1))).reshape(-1)
    test_x = get_test_x(n, mu=x_new.reshape(-1),  sigma=[0.25 for i in range(6)])
    print(f"{it+1}/{max_iter}: {best_f, train_y[-1].item()}")
    it += 1

1/20: (0.1, 0.059058841317892075)
2/20: (0.1, 0.0011186656774953008)
3/20: (0.12488909810781479, 0.12488909810781479)
4/20: (0.12488909810781479, 0.024153297767043114)
5/20: (0.12488909810781479, 0.03728562220931053)


[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/abauville/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/58/nr_cpjbj1sv1ckm03fv704qc0000gn/T/ipykernel_62521/1436507962.py", line 14, in <module>
    ei = EI(test_x)
  File "/Users/abauville/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/Users/abauville/opt/anaconda3/lib/python3.9/site-packages/botorch/utils/transforms.py", line 229, in decorated
    output = method(acqf, X, *args, **kwargs)
  File "/Users/abauville/opt/anaconda3/lib/python3.9/site-packages/botorch/acquisition/analytic.py", line 130, in forward
    posterior = self.model.posterior(
  File "/Users/abauville/opt/anaconda3/lib/python3.9/site-packages/botorch/models/gpytorch.py", line 146, in posterior
    mvn = self(X)
  File "/Users/abauville/opt/a

TypeError: object of type 'NoneType' has no len()

In [ ]:
ei

In [ ]:
train_y.shape

In [ ]:
model._train_targets[0]